In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options # opciones de chrome
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
from selenium.webdriver.support.ui import WebDriverWait   # para meter esperaras
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver import ActionChains # para hacer acciones con el ratón
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver.chrome.service import Service
import time
import selenium.webdriver.common.keys as Keys # para simular teclas
import pandas as pd

In [ ]:
#Opciones de chrome
opciones = webdriver.ChromeOptions()
opciones.add_argument('--start-maximized')
#opciones.add_argument('window-size=775,1400')
opciones.add_argument('--disable-extensions')
opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_argument('--no-sandbox')
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

opciones.add_argument('--incognito') # modo incognito
opciones.add_argument("--headless") #funciona sin abrir la página, funciona con menos recursos

In [ ]:
url = 'https://www.marphil.com/'
driver = webdriver.Chrome(options=opciones)
driver.get(url)

In [ ]:
#abro un desplegable donde voy a tener todas las categorías de la página
driver.find_element(By.CLASS_NAME, "elementor-menu-toggle").click()

In [ ]:
#busco las url de todas las categorías
#así mismo no voy a guardara las url de las categorías que tengan subcategoría
select = driver.find_element(By.ID, "menu-2-20de7bc")
lista_categorias = []
for i in range(2, 9):
    cat = select.find_element(By.XPATH, f'//*[@id="menu-2-20de7bc"]/li[{i}]')
    lista_ul = cat.find_elements(By.TAG_NAME, "ul")
    for ul in lista_ul:
        lista_a = ul.find_elements(By.TAG_NAME, "a")
        for href in lista_a:
            if 'has-submenu' not in href.get_attribute('class'):
                url_categoria = href.get_attribute("href")
                lista_categorias.append(url_categoria)

lista_categorias

In [ ]:
#una vez tengo la lista de las categorías, voy a recorrerla para obtener la información de cada producto
#además, al obtener las url de cada subcategoría, éstas no tienen más de una página de productos, 
#por lo que me ahorro tener que pulsar en la página siguiente

In [ ]:
# inicio un nuevo drive y una lista vacía en la que guardaré los productods
driver2 = webdriver.Chrome(options=opciones)
productos_marphil = []
#recorro la lista de categorías
for i in range(len(lista_categorias)):
    #spliteo la url de la categoría para obtener la categoría, subcategoría y tipo (en el caso en el que tenga)
    lista_cortada = lista_categorias[i].split("/")
    if len(lista_cortada) == 8:
        categoria = lista_cortada[4]
        subcategoria = lista_cortada[5]
        tipo = lista_cortada[6]
    elif len(lista_cortada) == 7:
        categoria = lista_cortada[4]
        subcategoria = lista_cortada[5]
        tipo = "no tiene"
    else:
        print(f"revisar: {lista_categorias[i]}") #esto lo puse inicialmente por si daba algún fallo, pero no fue necesario
    #hago la petición a la categoría
    driver2.get(lista_categorias[i])
    print(lista_categorias[i]) #printeo de control para saber por qué categoría voy
    #almaceno todos los productos de la categoría para recorrerlos en una lista anidada
    lista_productos = driver2.find_elements(By.CLASS_NAME, "jet-listing-grid__item")
    for producto in lista_productos:
        #almaceno la url del producto, su precio y su nombre
        nombre_producto = producto.find_element(By.CLASS_NAME, "jet-woo-builder-archive-product-title")
        url_producto = nombre_producto.find_element(By.TAG_NAME, "a").get_attribute("href")
        nombre = nombre_producto.text
        precio = producto.find_element(By.CLASS_NAME, "jet-woo-product-price").text
        #almaceno la información en un diccionario
        producto_dicc = {
            "categoria" : categoria,
            "subcategoria" : subcategoria,
            "tipo" : tipo,
            "nombre" : nombre, 
            "precio" : precio,
            "url" : url_producto,
            "tienda" : "Marphil"
        }
        print(f"añadido el producto: {nombre}") #printeo de control para ver el producto que se ha almacenado
        #apendeo los productos a la lista
        productos_marphil.append(producto_dicc)
    time.sleep(5)

In [ ]:
# Crear un DataFrame con la lista de productos_info
df_productos = pd.DataFrame(productos_marphil)

# Guardar el DataFrame en un archivo CSV
df_productos.to_csv("marphil.csv", index=False)